In [1]:
import tkinter as tk
from tkinter import ttk
import pandas as pd
import unicodedata
import zipfile
import os
import requests
import dask.dataframe as dd
import io
import codecs
import dbfread  # Será necessário instalar essa biblioteca via pip (pip install dbfread)
import tempfile
from pandastable import Table, TableModel

# Função para remover acentos e caracteres especiais
def remove_special_characters(text):
    return ''.join(c for c in unicodedata.normalize('NFKD', text) if not unicodedata.combining(c))

# Função para mostrar os resultados em uma tabela
def show_results_in_table(df):
    # Criar uma nova janela para exibir a tabela
    table_window = tk.Toplevel(root)
    table_window.title("Resultado do JOIN")

    # Criar o widget da tabela
    frame = ttk.Frame(table_window)
    frame.pack(fill=tk.BOTH, expand=True)

    pt = Table(frame, dataframe=df.compute())  # Usar .compute() para converter o DataFrame Dask em Pandas
    pt.show()

# Função para realizar o JOIN entre os DataFrames
def perform_join_and_display_ages():
    # Solicitar a URL do arquivo CSV da Bahia
    url = csv_url_entry.get()
    if not url:
        result_label.config(text="URL da Bahia não fornecida.")
        return

    # Solicitar a URL do arquivo ZIP
    zip_file_url = zip_url_entry.get()
    if not zip_file_url:
        result_label.config(text="URL do arquivo ZIP do Rio de Janeiro não fornecida.")
        return

    try:
        # Ler o arquivo CSV
        #df_url = dd.read_csv(url, delimiter=determinar_delimitador(url))
        df_url = dd.read_csv(url, delimiter=determinar_delimitador(url), low_memory=False)
    except Exception as e:
        error_message = f"Erro ao ler o arquivo CSV. Detalhes do erro: {str(e)}"
        result_label.config(text=error_message)
        print(error_message)
        return

    try:
        # Download do arquivo ZIP
        r = requests.get(zip_file_url, stream=True)
        temp_zip_file = tempfile.NamedTemporaryFile(delete=False)

        # Escrever o conteúdo do arquivo ZIP temporário
        with open(temp_zip_file.name, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)

        # Extrair o arquivo ZIP
        with zipfile.ZipFile(temp_zip_file.name, 'r') as z:
            dbf_files = [file for file in z.namelist() if file.endswith('.dbf')]
            if not dbf_files:
                raise ValueError("Não foi encontrado nenhum arquivo DBF no arquivo ZIP.")

            # Considerando o primeiro arquivo DBF encontrado no ZIP
            z.extract(dbf_files[0])
            dbf_file_path = os.path.join(os.getcwd(), dbf_files[0])

        # Lendo o arquivo DBF com a especificação de codificação
        with open(dbf_file_path, 'rb') as dbf_file:
            dbf_content = dbf_file.read()
            # Lendo o arquivo DBF sem usar BytesIO
            df_dbf = pd.DataFrame(iter(dbfread.DBF(dbf_file_path, encoding='latin1')))

            # Convertendo a coluna 'CD_MUN' para int64 no df_dbf
            df_dbf['CD_MUN'] = df_dbf['CD_MUN'].astype('int64')

    except Exception as e:
        error_message = f"Erro ao baixar, ler o arquivo ZIP ou realizar JOIN. Detalhes do erro: {str(e)}"
        result_label.config(text=error_message)
        print(error_message)
        return
    
    # Realizar o merge usando Dask
    join_type = join_type_var.get()  # Tipo de JOIN selecionado (inner, outer, left, right)
    merged_df = dd.merge(df_url, df_dbf, left_on='municipioIBGE', right_on='CD_MUN', how=join_type)
    
    # Mostrar os resultados em uma tabela
    show_results_in_table(merged_df)

    # Verificar a presença das colunas 'municipioIBGE' e 'CD_MUN' nos DataFrames originais
    if 'municipioIBGE' not in df_url.columns or 'CD_MUN' not in df_dbf.columns:
        error_message = "As colunas 'municipioIBGE' e 'CD_MUN' não estão presentes nos DataFrames originais."
        result_label.config(text=error_message)
        print(error_message)
        return
    
    # Realizar o merge usando Dask
    join_type = join_type_var.get()  # Tipo de JOIN selecionado (inner, outer, left, right)
    merged_df = dd.merge(df_url, df_dbf, left_on='municipioIBGE', right_on='CD_MUN', how=join_type)
    
    # Exibir a coluna 'AREA_KM2' após o merge se ela existir
    if 'AREA_KM2' in merged_df.columns:
        result_label.config(text="Coluna 'AREA_KM2' após o JOIN:\n" + str(merged_df['AREA_KM2']))
    else:
        result_label.config(text="A coluna 'AREA_KM2' não existe no DataFrame resultante.")

# Função para determinar o delimitador do arquivo CSV
def determinar_delimitador(url):
    r = requests.get(url)
    line = r.content.decode().splitlines()[0]
    delimiters = [',', ';', '\t']
    for delimiter in delimiters:
        if delimiter in line:
            return delimiter
    return ','  # Delimitador padrão é a vírgula

# Configuração da janela principal
root = tk.Tk()
root.title("App Join Dataframes")

# URL csv
csv_url_label = ttk.Label(root, text="URL do arquivo CSV:")
csv_url_entry = ttk.Entry(root, width=40)

# URL do arquivo ZIP
zip_url_label = ttk.Label(root, text="URL do arquivo ZIP:")
zip_url_entry = ttk.Entry(root, width=40)

# Tipo de JOIN
join_type_var = tk.StringVar()
join_type_label = ttk.Label(root, text="Tipo de JOIN:")
join_type_combobox = ttk.Combobox(root, textvariable=join_type_var, values=["inner", "outer", "left", "right"])

# Botão para realizar a operação de JOIN
join_button = ttk.Button(root, text="Carregar", command=perform_join_and_display_ages)

# Rótulo para exibir a coluna 'AREA_KM2' após o merge
result_label = ttk.Label(root, text="")

# Posicionamento dos widgets
csv_url_label.grid(row=0, column=0, sticky="w")
csv_url_entry.grid(row=0, column=1, columnspan=2, sticky="w")
zip_url_label.grid(row=1, column=0, sticky="w")
zip_url_entry.grid(row=1, column=1, columnspan=2, sticky="w")
join_type_label.grid(row=2, column=0, sticky="w")
join_type_combobox.grid(row=2, column=1, columnspan=2, sticky="w")
join_button.grid(row=3, column=0, columnspan=3, pady=10)
result_label.grid(row=4, column=0, columnspan=3, pady=10)

# Iniciar a interface gráfica
root.mainloop()

d:\Users\Eduarda\anaconda3\Lib\site-packages\dask\dataframe\io\csv.py:195: DtypeWarning: Columns (12,16,26,31,33,34,37,57,58) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
Exception in Tkinter callback
Traceback (most recent call last):
  File "d:\Users\Eduarda\anaconda3\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\Eduarda\AppData\Local\Temp\ipykernel_25484\1114389315.py", line 95, in perform_join_and_display_ages
    show_results_in_table(merged_df)
  File "C:\Users\Eduarda\AppData\Local\Temp\ipykernel_25484\1114389315.py", line 29, in show_results_in_table
    pt = Table(frame, dataframe=df.compute())  # Usar .compute() para converter o DataFrame Dask em Pandas
                                ^^^^^^^^^^^^
  File "d:\Users\Eduarda\anaconda3\Lib\site-packages\dask\base.py", line 310, in compute
    (result,) = compute(self, traverse=False, **kwargs)
    

d:\Users\Eduarda\anaconda3\Lib\site-packages\dask\dataframe\io\csv.py:195: DtypeWarning: Columns (12,26,31,34,37,58,62) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
